<a href="https://colab.research.google.com/github/formasys/classification_pneumonia/blob/master/pneumonia_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install tensorflow

In [ ]:
! pip install tqdm

In [ ]:
import os
import zipfile
import numpy as np
#import tensorflow as tf
import matplotlib.pyplot as plt
from glob import glob

from tqdm import tqdm_notebook
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import load_model, Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

%matplotlib inline
tf.__version__

'2.2.0'

In [ ]:
data_path='/content/drive/My Drive/Kaggle/pneumonia_data/chest_xray'

In [ ]:
train_dir = os.path.join(data_path, "train")
test_dir = os.path.join(data_path, "test")

In [ ]:
IMG_SHAPE = (224, 224,3)

In [ ]:
base_model = VGG16(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

In [ ]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
base_model.trainable = False

In [ ]:
base_model.output

<tf.Tensor 'block5_pool_2/MaxPool:0' shape=(None, 7, 7, 512) dtype=float32>

In [ ]:
# useful for getting number of classes
folders = glob(train_dir+'/*')

In [ ]:
folders

['/content/drive/My Drive/Kaggle/pneumonia_data/chest_xray/train/NORMAL',
 '/content/drive/My Drive/Kaggle/pneumonia_data/chest_xray/train/PNEUMONIA']

In [ ]:
# our layers - you can add more if you want
x = Flatten()(base_model.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=prediction)

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [ ]:
model.compile(  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(train_dir,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(test_dir,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
model.fit_generator(training_set, epochs=5, validation_data=test_set)

Epoch 1/5
 51/163 [========>.....................] - ETA: 25:29 - loss: 1.8626 - accuracy: 0.7230

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 117 could not be retrieved. It could be because a worker has died.
  UserWarning)


 98/163 [=================>............] - ETA: 15:13 - loss: 1.1194 - accuracy: 0.7892

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 85 could not be retrieved. It could be because a worker has died.
  UserWarning)


155/163 [===========================>..] - ETA: 1:52 - loss: 0.7996 - accuracy: 0.8302

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 35 could not be retrieved. It could be because a worker has died.
  UserWarning)


163/163 [==============================] - 2417s 15s/step - loss: 0.7861 - accuracy: 0.8299 - val_loss: 1.3641 - val_accuracy: 0.7340
Epoch 2/5
163/163 [==============================] - 114s 701ms/step - loss: 0.2269 - accuracy: 0.9059 - val_loss: 1.1765 - val_accuracy: 0.7837
Epoch 3/5
163/163 [==============================] - 117s 718ms/step - loss: 0.2054 - accuracy: 0.9145 - val_loss: 0.6913 - val_accuracy: 0.8510
Epoch 4/5
163/163 [==============================] - 115s 707ms/step - loss: 0.1972 - accuracy: 0.9189 - val_loss: 0.4982 - val_accuracy: 0.8734
Epoch 5/5
163/163 [==============================] - 115s 707ms/step - loss: 0.1781 - accuracy: 0.9314 - val_loss: 0.6848 - val_accuracy: 0.7756


In [ ]:
valid_loss, valid_accuracy = model.evaluate_generator(test_set)

In [ ]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.7756410241127014
